In [34]:
#必要なライブラリをインポート
import pandas as pd
import numpy as np

df_hiyoshi = pd.read_csv('suumo_hiyoshi01.csv', sep='\t', encoding='utf-16')
#df_dennennchoufu = pd.read_csv('suumo_dennennchoufu.csv', sep='\t', encoding='utf-16')
#df_tamagawa = pd.read_csv('suumo_tamagawa.csv', sep='\t', encoding='utf-16')
#df_oookayama = pd.read_csv('suumo_oookayam.csv', sep='\t', encoding='utf-16')
#df_meguro = pd.read_csv('suumo_meguro.csv', sep='\t', encoding='utf-16')
#df_musashikoyama = pd.read_csv('suumo_musashikoyama.csv', sep='\t', encoding='utf-16')
#df_musashikosugi = pd.read_csv('suumo_musashikosugi.csv', sep='\t', encoding='utf-16')

UnicodeError: UTF-16 stream does not start with BOM

In [42]:
#不要な列を削除
df_hiyoshi.drop(['Unnamed: 0', '立地2', '立地3', 'マンション名', '住所', '間取り'], axis=1, inplace=True)

KeyError: "labels ['マンション名' '住所' '間取り'] not contained in axis"

In [43]:
df_hiyoshi.head()

,立地1,立地2,立地3,築年数,建物高さ,階,賃料,管理費,"敷/礼/保証/敷引,償却",専有面積
0,東急東横線/元住吉駅 歩10分,東急東横線/日吉駅 歩12分,NaN,築13年,2階建,2階,9.2万円,4000円,9.2万円/-,34.75m2
1,東急東横線/日吉駅 歩13分,東急東横線/元住吉駅 歩13分,東急東横線/武蔵小杉駅 歩27分,築50年,2階建,2階,4.7万円,1000円,4.7万円/-,22.6m2
2,東急東横線/日吉駅 歩13分,東急東横線/元住吉駅 歩13分,東急東横線/武蔵小杉駅 歩27分,築50年,2階建,1階,4.7万円,1000円,4.7万円/-,22m2
3,東急東横線/日吉駅 歩5分,グリーンライン/日吉本町駅 歩15分,東急東横線/元住吉駅 歩24分,築9年,2階建,1階,12.3万円,6000円,12.3万円/12.3万円,42m2
4,東急東横線/日吉駅 歩19分,東急東横線/綱島駅 歩24分,グリーンライン/日吉本町駅 歩26分,築14年,3階建,3階,6.6万円,5000円,-/-,30.08m2


In [31]:
#立地を「路線+駅」と「徒歩〜分」に分割
splitted1 = df_hiyoshi['立地1'].str.split(' 歩', expand=True)
splitted1.columns = ['立地11', '立地12']

#その他費用を、「敷金」「礼金」「保証金」「敷引,償却」に分割
splitted4 = df['敷/礼/保証/敷引,償却'].str.split('/', expand=True)
splitted4.columns = ['敷金', '礼金', '保証金', '敷引,償却']

#「敷引,償却」をさらに「敷引」「償却」に分割
splitted5 = df['敷引,償却'].str.split('・', expand=True)
splitted5.columns = ['敷引', '償却']

#分割したカラムを結合
df = pd.concat([df, splitted1, splitted4, splitted5], axis=1)

#分割前のカラムは分析に使用しないので削除しておく
df.drop(['立地1','立地2','立地3','敷/礼/保証/敷引,償却','敷引,償却'], axis=1, inplace=True)

In [ ]:
#「賃料」がNAの行を削除
df = df.dropna(subset=['賃料'])

In [ ]:
#エンコードをcp932に変更しておく（これをしないと、replaceできない）
df['賃料'].str.encode('cp932')
df['敷金'].str.encode('cp932')
df['礼金'].str.encode('cp932')
df['保証金'].str.encode('cp932')
df['敷引'].str.encode('cp932')
df['償却'].str.encode('cp932')
df['管理費'].str.encode('cp932')
df['築年数'].str.encode('cp932')
df['専有面積'].str.encode('cp932')
df['立地12'].str.encode('cp932')

In [ ]:
#数値として扱いたいので、不要な文字列を削除
df['賃料'] = df['賃料'].str.replace(u'万円', u'')
df['敷金'] = df['敷金'].str.replace(u'万円', u'')
df['礼金'] = df['礼金'].str.replace(u'万円', u'')
df['保証金'] = df['保証金'].str.replace(u'万円', u'')
df['敷引'] = df['敷引'].str.replace(u'万円', u'')
df['償却'] = df['償却'].str.replace(u'万円', u'')
df['管理費'] = df['管理費'].str.replace(u'円', u'')
df['築年数'] = df['築年数'].str.replace(u'新築', u'0') #新築は築年数0年とする
df['築年数'] = df['築年数'].str.replace(u'築', u'')
df['築年数'] = df['築年数'].str.replace(u'年', u'')
df['専有面積'] = df['専有面積'].str.replace(u'm', u'')
df['立地12'] = df['立地12'].str.replace(u'分', u'')

In [ ]:
#splitted7['徒歩1'] = df['立地12']

In [ ]:
#「-」を0に変換
df['管理費'] = df['管理費'].replace('-',0)
df['敷金'] = df['敷金'].replace('-',0)
df['礼金'] = df['礼金'].replace('-',0)
df['保証金'] = df['保証金'].replace('-',0)
df['敷引'] = df['敷引'].replace('-',0)
df['敷引'] = df['敷引'].replace('実費',0) #「実費」と文字列が入っている場合がある
df['償却'] = df['償却'].replace('-',0)

#Noneを0に変換
df['償却'] = [0 if x is None else x for x in df['償却']]

In [ ]:
#文字列から数値に変換
df['賃料'] = pd.to_numeric(df['賃料'])
df['管理費'] = pd.to_numeric(df['管理費'])
df['敷金'] = pd.to_numeric(df['敷金'])
df['礼金'] = pd.to_numeric(df['礼金'])
df['保証金'] = pd.to_numeric(df['保証金'])
df['敷引'] = pd.to_numeric(df['敷引'])
df['償却'] = pd.to_numeric(df['償却'])
df['築年数'] = pd.to_numeric(df['築年数'])
df['専有面積'] = pd.to_numeric(df['専有面積'])
df['立地12'] = pd.to_numeric(df['立地12'])

In [ ]:
#単位を合わせるために、管理費以外を10000倍。
df['賃料'] = df['賃料'] * 10000
df['敷金'] = df['敷金'] * 10000
df['礼金'] = df['礼金'] * 10000
df['保証金'] = df['保証金'] * 10000
df['敷引'] = df['敷引'] * 10000
df['償却'] = df['償却'] * 10000

In [ ]:
#管理費は実質的には賃料と同じく毎月支払うことになるため、「賃料+管理費」を家賃を見る指標とする
df['賃料+管理費'] = df['賃料'] + df['管理費']

#敷金/礼金と保証金は同じく初期費用であり、どちらかが適用されるため、合計を初期費用を見る指標とする
df['敷/礼/保証金'] = df['敷金'] + df['礼金'] + df['保証金']

In [ ]:
#階を数値化。地下はマイナスとして扱う
splitted10 = df['階'].str.split('-', expand=True)
splitted10.columns = ['階1', '階2']
splitted10['階1'].str.encode('cp932')
splitted10['階1'] = splitted10['階1'].str.replace(u'階', u'')
splitted10['階1'] = splitted10['階1'].str.replace(u'B', u'-')
splitted10['階1'] = pd.to_numeric(splitted10['階1'])
df = pd.concat([df, splitted10], axis=1)

In [ ]:
#建物高さを数値化。地下は無視。
df['建物高さ'].str.encode('cp932')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下1地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下2地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下3地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下4地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下5地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下6地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下7地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下8地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'地下9地上', u'')
df['建物高さ'] = df['建物高さ'].str.replace(u'平屋', u'1')
df['建物高さ'] = df['建物高さ'].str.replace(u'階建', u'')
df['建物高さ'] = pd.to_numeric(df['建物高さ'])